In [1]:
import numpy as np
from numpy import inf
from mip import Model
from scipy.sparse import csr_matrix, vstack
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from scipy.optimize import linprog

In [2]:
def read_mps(file: str):
    """
    Reads a .mps and saves all the data of the MILP:

    min c^T * x

    s.t. b_l <= A*x <= b_u
          lb <=   x <= ub
                x_i integer if integrality[i] = 1
    """
    mdl = Model(solver_name="CBC")
    mdl.read(file)

    # model parameters
    num_vars = len(mdl.vars)
    num_cons = len(mdl.constrs)

    # variable types and bounds
    lb = np.zeros(num_vars)
    ub = np.inf*np.ones(num_vars)
    integrality = np.zeros(num_vars)
    for i, var in enumerate(mdl.vars):
        lb[i] = var.lb
        ub[i] = var.ub
        if var.var_type != "C":
            integrality[i] = 1

    # objective
    c = np.zeros(num_vars)
    for i, var in enumerate(mdl.vars):
        if var in mdl.objective.expr:
            c[i] = mdl.objective.expr[var]
    if mdl.sense != "MIN":
        c *= -1.0

    # constraint coefficient matrix
    b_l = -np.inf*np.ones((num_cons))
    b_u = np.inf*np.ones((num_cons))
    row_ind = []
    col_ind = []
    data    = []
    for i, con in enumerate(mdl.constrs):
        if con.expr.sense == "=":
            b_l[i] = con.rhs
            b_u[i] = con.rhs
        elif con.expr.sense == "<":
            b_u[i] = con.rhs
        elif con.expr.sense == ">":
            b_l[i] = con.rhs
        for j, var in enumerate(mdl.vars):
            if var in (expr := con.expr.expr):
                coeff = expr[var]
                row_ind.append(i)
                col_ind.append(j)
                data.append(coeff)
    A = csr_matrix((data, (row_ind, col_ind)), shape=(num_cons, num_vars))
    return c, b_l, A, b_u, lb, ub, integrality

def st_array(arr):
    arr[arr == -inf] = -1e6
    arr[arr == inf] = 1e6
    return arr

def mps_data(file: str):
    c, bl, A, bu, lb, ub, integrality = read_mps(file)
    c = st_array(c)
    bl = st_array(bl)
    bu = st_array(bu)
    lb = st_array(lb)
    ub = st_array(ub)

    Aeq = A[bl == bu]
    Aub = vstack([-A[bl != bu], A[bl != bu]])
    bub = np.concatenate([-bl[bl != bu], bu[bl != bu]])
    beq = bl[bl == bu]
    bound = np.array([lb, ub]).T
    
    return c, Aeq, beq, Aub, bub, bound, integrality

In [3]:
case1 = "afiro.mps"
case2 = "adlittle.mps"

In [4]:
c, Aeq, beq, Aub, bub, bound, integrality = mps_data(case2)

Coin0001I At line 1 NAME          ADLITTLE
Coin0001I At line 2 ROWS
Coin0001I At line 60 COLUMNS
Coin0001I At line 315 RHS
Coin0001I At line 335 ENDATA
Coin0002I Problem ADLITTLE has 56 rows, 97 columns and 383 elements
Coin0008I ADLITTLE read with 0 errors


In [5]:
def gbm1(c, G, h, A, b):
    # Create a new model
    m1 = gp.Model("m1")
    m1.setParam('OutputFlag', 0)
    # Create variables
    x = m1.addMVar(shape=len(c), vtype=GRB.CONTINUOUS, name="x")
    m1.setObjective(c @ x, GRB.MINIMIZE)

    # Add constraints
    m1.addConstr(G @ x <= h, name="ic")
    m1.addConstr(A @ x == b, name="ec")
    m1.update()
    m1.optimize()
    return m1

In [6]:
%%timeit -r5 -n100
res = linprog(c, A_ub=Aub, A_eq=Aeq, b_ub=bub, b_eq=beq, bounds=bound)

2.75 ms ± 207 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)


In [7]:
%%timeit -r5 -n100
gbm1(c, Aub, bub, Aeq, beq)

Restricted license - for non-production use only - expires 2023-10-25
2.89 ms ± 198 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)
